In [1]:
%matplotlib inline

# 592B, Class 8.1 (10/22/19).  Midterm Review: how do you compute a spectrum/spectrogram?

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
from scipy import fftpack
from scipy import signal
from ipywidgets import interactive
from IPython.display import Audio, display

## Questions I want you to be able to answer

- Why is the computation of the spectrum affected by the "window length" over which it is computed, and how is it affected?
- Why and how does the choice of "window" over the waveform affect how "wide" peaks in the spectrum are?
- Why do you get negative frequencies when you do a Fourier Transform?

## Warm-up: hands-on spectrogram tutorial

Clone the [Github repository here](https://github.com/drammock/spectrogram-tutorial) and work through it. Try using a different wave file than the one used as a sample. Also compute a spectrogram/spectrograms using Praat for comparison. And try computing a narrow-band spectrogram, too.

Some things to think about and answer (besides thinking about big questions above):
- What library and function is being used to compute the spectrogram here and how do you find out more about it?
- What choices involved in computing a spectrogram also come up for computing a spectrum? What choices only come up for computing a spectrogram and not the spectrum?
- What would ringing look like in the spectrum?
- How do you compute a wide-band spectrogram? How do you compute a narrow-band spectrogram?

---

## Intro 2: windowing/leakage tutorial
    
Work through the Elegant SciPy tutorial section on [windowing](https://www.safaribooksonline.com/library/view/elegant-scipy/9781491922927/ch04.html#windowing). (You can copy and paste code from there into this notebook.)

Some things to think about/answer:
- What happens to the spectrum when you change the width of the rectangular pulse?
- Can you write a function so you can easily vary the width of the rectangular pulse and make the two plots?
- Why do we do windowing?
- How can you window with a Gaussian window rather than a Kaiser window?
- How is windowing for computing the spectrum related to windowing in computing the spectrogram?


## Intro 2.5: Convolution. What really is windowing anyway?

Last time we said that windowing, filtering, and smoothing, are the same operations from a mathematical perspective. That operation is called **convolution**. The convolution of two functions $f(t)$ and $g(t)$ is defined as: 

$ f * g = \int_{-\infty}^{\infty} f(\tau)g(t-\tau)d\tau $

The motto that goes with this is: flip and shift and compute the overlapping area.

Here are some examples:

A rectangular pulse with itself:
![Animation box](https://upload.wikimedia.org/wikipedia/commons/6/6a/Convolution_of_box_signal_with_itself2.gif)

![Animation rect](https://upload.wikimedia.org/wikipedia/commons/c/c6/Convolucion_Funcion_Pi.gif)

A rectangular pulse with a spiky function:
![Animation spiky](https://upload.wikimedia.org/wikipedia/commons/4/42/Convolucion_de_entrada_con_respuesta_al_impulso.gif)

![Animation spiky](https://upload.wikimedia.org/wikipedia/commons/6/67/Convolution_of_spiky_function_with_box.gif)


And a Gaussian with a Gaussian:

![Gaussian convolution](https://upload.wikimedia.org/wikipedia/commons/9/9c/Convolution_Animation_%28Gaussian%29.gif)

If we look at the [source code for the Gaussian window function scipy.signal.gaussian](https://github.com/scipy/scipy/blob/v1.0.0/scipy/signal/windows.py#L1159-L1219), it's basically just this:

```python
if _len_guards(M):
        return np.ones(M)
    M, needs_trunc = _extend(M, sym)

    n = np.arange(0, M) - (M - 1.0) / 2.0
    sig2 = 2 * std * std
    w = np.exp(-n ** 2 / sig2)

return _truncate(w, needs_trunc)

```
And in the Elegant Scipy tutorial, the windowing happens with this:

```python
win = np.kaiser(len(t), 5)
X_win = fftpack.fft(x * win)

plt.plot(fftpack.fftfreq(len(t)), np.abs(X_win))
plt.ylim(0, 190);
```

And there is a beautiful theorem that **convolution in the time domain is multiplication in the time domain; convolution in the frequency domain is multiplication in the frequency domain.** See Osgood notes, Chapter 3 for more details.

## Intro 3: Time-limited signals and window length

Do the exercises I found [here](https://www.gaussianwaves.com/2011/01/fft-and-spectral-leakage-2/). Note that the code is in Matlab; you'll need to port to Python and modify as necessary.

Things to think about and answer:

- What causes spectral leakage?

## Intro 4: theoretical underpinnings

Remember that we ended up with the Fourier series of $g(t)$ defined as $T \rightarrow \infty$, expressed as a double integral:

\begin{equation}
g(t) = \int_{-\infty}^{\infty} \int_{-\infty}^{\infty}e^{-2\pi ift}g(t)dt e^{2\pi ift}df
\end{equation}

and then we derived $\mathcal{F}g$, where $\mathcal{F}g$ is defined as the Fourier transform of a function $g(t)$:

\begin{equation}
\mathcal{F}g = \int_{-\infty}^{\infty}e^{-2\pi ift} g(t)dt      
\end{equation}

and the inverse Fourier transform $\mathcal{F}^{-1}(t)$ of a function $\mathcal{F}g(f)$ is: 

\begin{equation}
g(t) = \int_{-\infty}^{\infty}e^{2\pi ift} \mathcal{F}gdf      
\end{equation}

Then we briefly introduced the **discrete Fourier transform (DFT)**: this is what we use when we're computing the Fourier Transform in python because we are working with *digital* signals. 



## The Discrete Fourier Transform

Today we're going to work on understanding the DFT a little more, because it will help us understand what's going on when we invoke `fftpack.fft` and have a deeper understanding of the answers to the review questions above. The derivation here follows Osgood Chapter 6.

Our goal is to find a discrete version of $\mathcal{F}g(f)$, the Fourier transform of a function $g(t)$. (Note: by writing $\mathcal{F}g(f)$, we mean that the function $\mathcal{F}g$, the Fourier transform of $g(t)$, is a function of frequency, $f$. We start with $g(t)$, which is a function of time $t$, but once we take the Fourier transform of $g(t)$, $\mathcal{F}g$, we have a function of frequency $f$.). 

To do this, we need to find three things:
- A discrete version of $g(t)$ that reasonably approximates $g(t)$
- A discrete version of $\mathcal{F}g(f)$ that reasonably approximates $\mathcal{F}g(f)$
- A way in which these two discrete versions are related, which approximates the relation between the continuous versions

We start by assuming that $g(t)$ is:
- *time-limited*, meaning that $g(t)$ is zero outside of $0\leq t \leq L$, where $L$ is the length of the signal (in time)
- *band-limited*, meaning that $\mathcal{F}g(f)$ vanishes outside of $0 \lt f \lt 2B$, where $B$ is the *bandwith* of the signal.


By the Sampling Theorem, if we sample $g(t)$ at the Nyquist rate of $2B$ samples/second, we can reconstruct $g(t)$ perfectly. This sampled version of $g(t)$, call it $g_{discrete}(t)$, is just a list of $N$ sampled values:

$$ g(t_0), g(t_1), \ldots, g(t_{N-1}) $$,

where $N=2BL$ and the timepoints are evenly spaced apart by $\frac{1}{2B}$.

***In-class exercise: Why is $N=2BL$ and why does $\Delta t_n = \frac{1}{2B}$?***

---


### The Dirac comb

We can re-express $g_{discrete}(t)$ using the Dirac comb $III(t)$, defined as:

$$III(t) = \displaystyle\sum_{n=0}^{N-1} \delta(t-t_n) $$

All this is is a "train" of pulses, a comb of "lollipops" with amplitude 1, where the pulses occur exactly at the sampled points $t_0, t_1, \ldots, t_{N-1}$.

Here's an image of a Dirac comb from Wikipedia. In our case, $T = 1/2B$.

<img alt = "Dirac comb plot" src="https://upload.wikimedia.org/wikipedia/commons/4/49/Dirac_comb.svg" width="300"  />


And here's an example of [one way to define a Dirac comb function](https://scipython.com/book/chapter-6-numpy/examples/a-comb-function/), from [scipython.com](https://scipython.com).


In [4]:
N, n = 101, 5
def f(i):
    return (i % n == 0) * 1

comb = np.fromfunction(f, (N,), dtype=int)
print(comb)


[1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1]


***In-class discussion: Does the Dirac comb form an orthogonal basis set?***

---

Using $III(t)$, we can now express $g_{discrete}(t)$ as:

$$g_{discrete}(t) = g(t) \displaystyle\sum_{n=0}^{N-1} \delta(t-t_n) = \displaystyle\sum_{n=0}^{N-1} g(t) \delta(t-t_n) $$

And the Fourier transform of $g_{discrete}(t)$ is:

\begin{equation}
\mathcal{F}g_{discrete}(f) = \displaystyle\sum_{n=0}^{N-1} = \mathcal{F}g(t_n) \delta(t-t_n) = \displaystyle\sum_{n=0}^{N-1} g(t_n) e^{-2\pi ift_n}      
\end{equation}

This gives us the continuous Fourier transform of the sampled version of $g(t)$.

Now let's think about $g(t)$ in the frequency domain. Remember by assumption that $g(t)$ is time-limited so $g(t)$ is zero outside of $0\leq t \leq L$, where $L$ is the length of the signal (in time). So we can apply the Sampling Theorem to reconstruct $\mathcal{F}g(f)$ in the frequency domain. The sampling rate we need (the Nyquist rate) for perfect reconstruction is $L$ samples/Hz and the spacing between sampling points is $1/L$.

Since $\mathcal{F}g(f)$ is band-limited by assumption and  vanishes outside of $0 \lt f \lt 2B$, we sample $\mathcal{F}g(f)$ over $0 \lt f \lt 2B$, with points $1/L$ Hz apart. 

---

***In-class exercise: Why is the sampling rate $L$ samples/Hz and why is the interval between sampling points $1/L$ Hz? What is the total number of sampling points, $N$***

---

This sampled version of $\mathcal{F}g(f)$, call it $\mathcal{F}g_{discrete}(f)$, is just a list of $N$ sampled values, of the form $m/L$, where $m$ is a non-negative integer:

$$ f_0=0,\, f_1 = \frac{1}{L},\, \ldots, f_{N-1} = \frac{N-1}{L} $$,

And if we want the discrete version of $g_{discrete}(t)$, then we want to take $[\mathcal{F}(g_{discrete})](f)$, call this $F(f)$ for short.

Taking our definition of the Fourier transform of $g_{discrete}(t)$,

$$\mathcal{F}g(t_n) \delta(t-t_n) = \displaystyle\sum_{n=0}^{N-1} g(t_n) e^{-2\pi ift_n}$$

this will give us the list:

$$ F(f_0) = \displaystyle\sum_{n=0}^{N-1} g(t_n) e^{-2\pi if_0t_n}, \ldots, F(f_{N-1})=\displaystyle\sum_{n=0}^{N-1} g(t_n) e^{-2\pi if_{N-1}t_n}  $$

And so now we have a way to go from $g_{discrete}(t)$ to $\mathcal{F}g_{discrete}(f)$, for each $m$ from $m=0$ to $m=N-1$:

$$F(f_m) = \displaystyle\sum_{n=0}^{N-1} g(t_n) e^{-2\pi if_mt_n} $$

Recalling that $t_n = \frac{n}{2B}$ and $f_m = \frac{m}{L}$ and $N=2BL$, we can re-write this as:

$$F(f_m) = \displaystyle\sum_{n=0}^{N-1} g(t_n) e^{-2\pi inm/N} $$

***In-class exercise: derive our final expression of $F(f_m)$.***

---

At this point, let's come back to one of our starting questions and discuss. You should have more insight on this now! What is the "grid" spacing in the time-domain? The frequency domain? How are they related? 

> Why is the computation of the spectrum affected by the "window length" over which it is computed, and how is it affected?

---

## Positive and negative frequencies (Osgood 2010, p. 260)

Given our discrete Fourier transform $[\mathcal{F}(g_{discrete})](f)$, call this $F(f)$:

$$F(f_m) = \displaystyle\sum_{n=0}^{N-1} g(t_n) e^{-2\pi inm/N} $$

it turns out that the spectrum *splits* at $N/2$. See Osgood (2010) for the derivation, but due to some periodicity relations:

$$ F[\frac{N}{2} + 1] = \overline{F[\frac{N}{2} - 1]}$$ 
$$ F[\frac{N}{2} + 2] = \overline{F[\frac{N}{2} - 2]}$$ 
$$ \vdots$$

***In-class exercise: What is F[0]? What do the periodicity relations mean geometrically?***

So because of this, the convention is to say, for a spectrum indexed from 0 to $N-1$:
- The frequencies from $m=1$ to $m= N/2-1$ are the "positive" frequencies
- The frequencies from $m=N/2+1$ to $m= N-1$ are the "negative" frequencies

For a real signal, all the information you need is in the positive frequencies and the first component $F[0]$.

